In [12]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import warnings

warnings.filterwarnings('ignore')

def gen_serve_dist_bar(path, player):
    # Load the data
    events = pd.read_csv(path)
    events['pointWonBy'] = events.groupby('pointNumber')['pointWonBy'].bfill()
    
    # Filter the events to get only serves for player1
    filtered_events = events[
        (events['shotInRally'] == 1) &
        (events['serverName'] == player) &
        ((events['firstServeIn'] == 1.0) | (events['secondServeIn'] == 1.0))
    ]
    
    valid_placements = ['Wide', 'T', 'Body']
    filtered_events = filtered_events[filtered_events['serveInPlacement'].isin(valid_placements)]
    filtered_events['Zone'] = filtered_events['side'] + " " + filtered_events['serveInPlacement']

    serve_counts = filtered_events.groupby(['side', 'Zone']).size()
    side_totals = serve_counts.groupby(level=0).sum()

    zone_percentages = (serve_counts / side_totals) * 100
    zone_percentages = zone_percentages.round().astype(int)
    zone_percentages = zone_percentages.reset_index()

    serve_dist = pd.DataFrame({
        "Zone": zone_percentages['Zone'],
        "Percentage": zone_percentages[0].astype(str) + '%'
    })

    serve_dist['Server'] = [player, "", "", "", "", ""]
    
    serve_dist_bar_json = serve_dist.to_json(orient='records')
    with open('serve_dist_bar.json', 'w') as f:
        f.write(serve_dist_bar_json)

player = "Spencer Johnson"
path = "../../Match CSVs/Shot_Visuals_SpencerJohnson_OscarPintoSansano.csv"
gen_serve_dist_bar(path, player)